In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig
import time
import csv
import sys
import plotly.express as px

In [2]:
df = pd.read_csv("/Users/hamin/bithumb_bot_data/include/up-btc/upbit-2018-08-btc-krw.csv")

In [3]:
df.dtypes

timestamp     object
quantity     float64
price          int64
fee          float64
amount         int64
side           int64
dtype: object

In [4]:
## converting timestamp column type 'object' to 'datetime'
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df.dtypes

timestamp    datetime64[ns]
quantity            float64
price                 int64
fee                 float64
amount                int64
side                  int64
dtype: object

In [5]:
## converting timestamp to detailed date column
df['timestamp_year_month_day']       = df['timestamp'].dt.date         # YYYY-MM-DD(문자)
df['timestamp_year']       = df['timestamp'].dt.year         # 연(4자리숫자)
df['timestamp_month']      = df['timestamp'].dt.month        # 월(숫자)
df['timestamp_month_name'] = df['timestamp'].dt.month_name() # 월(문자)

df['timestamp_day']        = df['timestamp'].dt.day          # 일(숫자)
df['timestamp_time']       = df['timestamp'].dt.time         # HH:MM:SS(문자)
df['timestamp_hour']       = df['timestamp'].dt.hour         # 시(숫자)
df['timestamp_minute']     = df['timestamp'].dt.minute       # 분(숫자)
df['timestamp_second']     = df['timestamp'].dt.second       # 초(숫자)

In [6]:
df

,timestamp,quantity,price,fee,amount,side,timestamp_year_month_day,timestamp_year,timestamp_month,timestamp_month_name,timestamp_day,timestamp_time,timestamp_hour,timestamp_minute,timestamp_second
0,2018-08-08 00:00:00,0.001427,8127000,5.79,11591,1,2018-08-08,2018,8,August,8,00:00:00,0,0,0
1,2018-08-08 00:00:00,0.004146,8127000,16.84,33679,1,2018-08-08,2018,8,August,8,00:00:00,0,0,0
2,2018-08-08 00:11:00,0.008111,8135000,32.99,65947,1,2018-08-08,2018,8,August,8,00:11:00,0,11,0
3,2018-08-08 00:23:00,0.000150,8124000,0.60,1214,1,2018-08-08,2018,8,August,8,00:23:00,0,23,0
4,2018-08-08 00:31:00,0.010259,8141000,41.76,83478,1,2018-08-08,2018,8,August,8,00:31:00,0,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12210,2018-08-31 23:28:00,0.022331,7782000,86.88,-173867,0,2018-08-31,2018,8,August,31,23:28:00,23,28,0
12211,2018-08-31 23:29:00,0.089700,7782000,349.02,-698395,0,2018-08-31,2018,8,August,31,23:29:00,23,29,0
12212,2018-08-31 23:31:00,1.919860,7785000,7473.05,-14953580,0,2018-08-31,2018,8,August,31,23:31:00,23,31,0
12213,2018-08-31 23:31:00,0.122936,7785000,478.52,-957535,0,2018-08-31,2018,8,August,31,23:31:00,23,31,0


In [7]:
## creating 'day - amount' connection
group_day_amount = df['amount'].groupby(df['timestamp_year_month_day'])

In [8]:
group_day_amount.size()

timestamp_year_month_day
2018-08-08    1396
2018-08-09    1492
2018-08-10     980
2018-08-11    1150
2018-08-12     440
2018-08-13     454
2018-08-14     809
2018-08-15     636
2018-08-16     562
2018-08-17     611
2018-08-18     634
2018-08-19     208
2018-08-20     325
2018-08-21     268
2018-08-22     237
2018-08-23     245
2018-08-24     129
2018-08-25     163
2018-08-26     185
2018-08-27     219
2018-08-28     226
2018-08-29     252
2018-08-30     309
2018-08-31     285
Name: amount, dtype: int64

In [9]:
group_day_amount

In [10]:
## 일별 총 손익량
group_day_amount.sum()

timestamp_year_month_day
2018-08-08    56402704
2018-08-09   -35741956
2018-08-10   -10239554
2018-08-11    15657118
2018-08-12      904614
2018-08-13    -4297854
2018-08-14     -931164
2018-08-15    19984522
2018-08-16    -6059726
2018-08-17     7548718
2018-08-18      250396
2018-08-19      346330
2018-08-20      546843
2018-08-21     -779107
2018-08-22   -11523720
2018-08-23    12836750
2018-08-24   -20880077
2018-08-25    21216488
2018-08-26    -5165021
2018-08-27     4862641
2018-08-28      658551
2018-08-29   -26201732
2018-08-30    25800366
2018-08-31   -30911509
Name: amount, dtype: int64

In [11]:
## creating 'day - earning amount' connection
df2 = df.copy()
df2 =df2[df2['amount'] > 0]
group_day_amount_positive = df2['amount'].groupby(df2['timestamp_year_month_day'])


In [12]:
# 일별 수익 transaction 수
group_day_amount_positive.size()

timestamp_year_month_day
2018-08-08    874
2018-08-09    984
2018-08-10    532
2018-08-11    864
2018-08-12    259
2018-08-13    286
2018-08-14    574
2018-08-15    392
2018-08-16    377
2018-08-17    397
2018-08-18    391
2018-08-19    137
2018-08-20    220
2018-08-21    179
2018-08-22    141
2018-08-23    141
2018-08-24     87
2018-08-25     90
2018-08-26    101
2018-08-27    140
2018-08-28    109
2018-08-29    138
2018-08-30    182
2018-08-31    150
Name: amount, dtype: int64

In [13]:
# 일별 수익량
group_day_amount_positive.sum()

timestamp_year_month_day
2018-08-08    1444902260
2018-08-09    1103105278
2018-08-10     672667548
2018-08-11     870248654
2018-08-12     367116431
2018-08-13     390649583
2018-08-14     680850521
2018-08-15     504132688
2018-08-16     466611113
2018-08-17     475565347
2018-08-18     549137075
2018-08-19     166214906
2018-08-20     376673678
2018-08-21     225309271
2018-08-22     305520462
2018-08-23     244221449
2018-08-24     155295014
2018-08-25     162403905
2018-08-26     139189177
2018-08-27     198029879
2018-08-28     253258137
2018-08-29     304913298
2018-08-30     325679457
2018-08-31     312982097
Name: amount, dtype: int64

In [14]:
## creating 'day - loss amount' connection
df3 = df.copy()
df3 = df3[df3['amount'] < 0]
group_day_amount_negative = df3['amount'].groupby(df3['timestamp_year_month_day'])

In [15]:
# 일별 손해 transaction 수
group_day_amount_negative.size()

timestamp_year_month_day
2018-08-08    522
2018-08-09    506
2018-08-10    448
2018-08-11    284
2018-08-12    180
2018-08-13    167
2018-08-14    235
2018-08-15    244
2018-08-16    185
2018-08-17    213
2018-08-18    243
2018-08-19     71
2018-08-20    105
2018-08-21     89
2018-08-22     96
2018-08-23    104
2018-08-24     42
2018-08-25     73
2018-08-26     84
2018-08-27     77
2018-08-28    117
2018-08-29    114
2018-08-30    127
2018-08-31    135
Name: amount, dtype: int64

In [16]:
# 일별 손해량
group_day_amount_negative.sum()

timestamp_year_month_day
2018-08-08   -1388499556
2018-08-09   -1138847234
2018-08-10    -682907102
2018-08-11    -854591536
2018-08-12    -366211817
2018-08-13    -394947437
2018-08-14    -681781685
2018-08-15    -484148166
2018-08-16    -472670839
2018-08-17    -468016629
2018-08-18    -548886679
2018-08-19    -165868576
2018-08-20    -376126835
2018-08-21    -226088378
2018-08-22    -317044182
2018-08-23    -231384699
2018-08-24    -176175091
2018-08-25    -141187417
2018-08-26    -144354198
2018-08-27    -193167238
2018-08-28    -252599586
2018-08-29    -331115030
2018-08-30    -299879091
2018-08-31    -343893606
Name: amount, dtype: int64

In [17]:
# 일별 절대값 총합
absolute_negative_sum=group_day_amount_negative.sum()
absolute_negative_sum = absolute_negative_sum * (-1)
absolute_total_amout=absolute_negative_sum+group_day_amount_positive.sum()
absolute_total_amout

timestamp_year_month_day
2018-08-08    2833401816
2018-08-09    2241952512
2018-08-10    1355574650
2018-08-11    1724840190
2018-08-12     733328248
2018-08-13     785597020
2018-08-14    1362632206
2018-08-15     988280854
2018-08-16     939281952
2018-08-17     943581976
2018-08-18    1098023754
2018-08-19     332083482
2018-08-20     752800513
2018-08-21     451397649
2018-08-22     622564644
2018-08-23     475606148
2018-08-24     331470105
2018-08-25     303591322
2018-08-26     283543375
2018-08-27     391197117
2018-08-28     505857723
2018-08-29     636028328
2018-08-30     625558548
2018-08-31     656875703
Name: amount, dtype: int64

In [18]:
## 일별 계산 table 생성
dfs = [group_day_amount.size(), group_day_amount.sum(), group_day_amount_positive.size(), group_day_amount_positive.sum(), group_day_amount_negative.size(), group_day_amount_negative.sum(), absolute_total_amout, absolute_negative_sum]
dfs

[timestamp_year_month_day
 2018-08-08    1396
 2018-08-09    1492
 2018-08-10     980
 2018-08-11    1150
 2018-08-12     440
 2018-08-13     454
 2018-08-14     809
 2018-08-15     636
 2018-08-16     562
 2018-08-17     611
 2018-08-18     634
 2018-08-19     208
 2018-08-20     325
 2018-08-21     268
 2018-08-22     237
 2018-08-23     245
 2018-08-24     129
 2018-08-25     163
 2018-08-26     185
 2018-08-27     219
 2018-08-28     226
 2018-08-29     252
 2018-08-30     309
 2018-08-31     285
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-08-08    56402704
 2018-08-09   -35741956
 2018-08-10   -10239554
 2018-08-11    15657118
 2018-08-12      904614
 2018-08-13    -4297854
 2018-08-14     -931164
 2018-08-15    19984522
 2018-08-16    -6059726
 2018-08-17     7548718
 2018-08-18      250396
 2018-08-19      346330
 2018-08-20      546843
 2018-08-21     -779107
 2018-08-22   -11523720
 2018-08-23    12836750
 2018-08-24   -20880077
 2018-08-25    21216488
 2018-0

In [19]:
day_df_final = reduce(lambda left,right: pd.merge(left,right,on='timestamp_year_month_day'), dfs)
day_df_final.columns = [ "daily transaction", "daily income", "daily Yield transaction", "daily Yield", "daily loss transaction", "daily loss", "daily absolute amount", "daily absolute negative amount"]
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-08-08,1396,56402704,874,1444902260,522,-1388499556,2833401816,1388499556
2018-08-09,1492,-35741956,984,1103105278,506,-1138847234,2241952512,1138847234
2018-08-10,980,-10239554,532,672667548,448,-682907102,1355574650,682907102
2018-08-11,1150,15657118,864,870248654,284,-854591536,1724840190,854591536
2018-08-12,440,904614,259,367116431,180,-366211817,733328248,366211817
2018-08-13,454,-4297854,286,390649583,167,-394947437,785597020,394947437
2018-08-14,809,-931164,574,680850521,235,-681781685,1362632206,681781685
2018-08-15,636,19984522,392,504132688,244,-484148166,988280854,484148166
2018-08-16,562,-6059726,377,466611113,185,-472670839,939281952,472670839


In [20]:
print(day_df_final.asfreq('D'))

                          daily transaction  daily income  \
timestamp_year_month_day                                    
2018-08-08                             1396      56402704   
2018-08-09                             1492     -35741956   
2018-08-10                              980     -10239554   
2018-08-11                             1150      15657118   
2018-08-12                              440        904614   
2018-08-13                              454      -4297854   
2018-08-14                              809       -931164   
2018-08-15                              636      19984522   
2018-08-16                              562      -6059726   
2018-08-17                              611       7548718   
2018-08-18                              634        250396   
2018-08-19                              208        346330   
2018-08-20                              325        546843   
2018-08-21                              268       -779107   
2018-08-22              

In [21]:
day_df_final.to_csv("/Users/hamin/bithumb_bot_data/upbit-2018-07-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')


In [24]:
# 뒤집기.
day_df_final[::-1].to_csv("/Users/hamin/bithumb_bot_data/upbit-2018-07-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')

In [23]:
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-08-08,1396,56402704,874,1444902260,522,-1388499556,2833401816,1388499556
2018-08-09,1492,-35741956,984,1103105278,506,-1138847234,2241952512,1138847234
2018-08-10,980,-10239554,532,672667548,448,-682907102,1355574650,682907102
2018-08-11,1150,15657118,864,870248654,284,-854591536,1724840190,854591536
2018-08-12,440,904614,259,367116431,180,-366211817,733328248,366211817
2018-08-13,454,-4297854,286,390649583,167,-394947437,785597020,394947437
2018-08-14,809,-931164,574,680850521,235,-681781685,1362632206,681781685
2018-08-15,636,19984522,392,504132688,244,-484148166,988280854,484148166
2018-08-16,562,-6059726,377,466611113,185,-472670839,939281952,472670839
